### UK MOT Dataset
***1. Load dataset***

- Load neccessary packages
- Load dataset into dataframe

In [1]:
#Load Packages
import pandas as pd
import numpy as np
import seaborn as sns

#load csv in dataframe
mot_data = pd.read_csv('mot2023.csv', delimiter='|')

mot_data.head()

,test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
0,1994821045,838565361,2023-01-02,4,NT,P,179357.0,NW,TOYOTA,PRIUS +,WHITE,HY,1798.0,2016-06-17
1,358005195,484499974,2023-01-01,4,NT,P,300072.0,B,TOYOTA,PRIUS,RED,HY,1500.0,2008-09-13
2,773392437,53988366,2023-01-02,4,NT,PRS,307888.0,HA,TOYOTA,PRIUS,GREY,HY,1497.0,2010-01-15
3,133665147,606755010,2023-01-02,4,NT,F,65810.0,SE,TOYOTA,PRIUS,SILVER,HY,1497.0,2007-03-28
4,656743571,606755010,2023-01-02,4,RT,P,65810.0,SE,TOYOTA,PRIUS,SILVER,HY,1497.0,2007-03-28


***2. Standardize data formats***
- check and update dataset to ensure all variables have correct data types
- ensure that all date and time fields are in a consistent format and recognized as datetime objects.
- trim whitespaces from strings

In [2]:
#check for duplicates
print(mot_data.duplicated().sum())
#start by checking the data types
print(mot_data.dtypes)

0
test_id                int64
vehicle_id             int64
test_date             object
test_class_id          int64
test_type             object
test_result           object
test_mileage         float64
postcode_area         object
make                  object
model                 object
colour                object
fuel_type             object
cylinder_capacity    float64
first_use_date        object
dtype: object


In [3]:
#convert and standardize the dates
mot_data['test_date'] = pd.to_datetime(mot_data['test_date'], format='%Y-%m-%d')
#mot_data['first_use_date'] = pd.to_datetime(mot_data['first_use_date'], format='%Y-%m-%d')

#convert to string to confirm valid string
mot_data['test_type'] = mot_data['test_type'].astype(str)
mot_data['postcode_area'] = mot_data['postcode_area'].astype(str)

#consistent formatting
mot_data['make'] = mot_data['make'].str.strip().str.upper()
mot_data['model'] = mot_data['model'].str.strip().str.upper()

#categorical variables
mot_data['colour'] = mot_data['colour'].astype('category')
mot_data['fuel_type'] = mot_data['fuel_type'].astype('category')
mot_data['test_result'] = mot_data['test_result'].astype('category')

#confirm the data types
print(mot_data.dtypes)


test_id                       int64
vehicle_id                    int64
test_date            datetime64[ns]
test_class_id                 int64
test_type                    object
test_result                category
test_mileage                float64
postcode_area                object
make                         object
model                        object
colour                     category
fuel_type                  category
cylinder_capacity           float64
first_use_date               object
dtype: object


Note: `first_use_date` standardization failed with <span style="color:red">OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 211-06-14 00:00:00, at position 453727.</span>

- Next step is to investigate the invalid dates in this variable and drop the invalid dates if it's not a lot as this is an important variable

In [4]:
# Display rows with invalid dates
invalid_dates = mot_data[pd.to_datetime(mot_data['first_use_date'], format='%Y-%m-%d', errors='coerce').isna() & mot_data['first_use_date'].notna()]
print(invalid_dates)

             test_id  vehicle_id  test_date  test_class_id test_type  \
453727     547414349  1167558481 2023-01-05              4        NT   
3966460   1418143843  1135810061 2023-02-02              4        NT   
6727297   1777168585  1366277257 2023-02-24              4        NT   
8872607   1975215653  1060830825 2023-03-13              4        NT   
9426118   1623521893   805711097 2023-03-16              4        NT   
10903966  1508813875  1060871885 2023-03-27              4        NT   
12336596  1347132577   576919031 2023-04-11              4        NT   
12811065   446091971  1453070495 2023-04-15              4        NT   
13297651   104009305   655960856 2023-04-20              4        NT   
13418777   442298581    74536149 2023-04-22              4        NT   
14829798  1036745685  1187798365 2023-05-10              4        NT   
14830798  1230131787  1187798365 2023-05-10              4        NT   
15364193  1640141555   307821629 2023-05-15              1      

In [5]:
#get count of the invalid dates and if it is not a lot drop it
invalid_dates_count = invalid_dates.shape[0]
print(invalid_dates_count)

31


In [6]:
# Convert invalid dates to NaT
mot_data['first_use_date'] = pd.to_datetime(mot_data['first_use_date'], format='%Y-%m-%d', errors='coerce')

# drop rows with NaT in 'first_use_date'
mot_data = mot_data.dropna(subset=['first_use_date'])

#run initial conversion that failed
mot_data['first_use_date'] = pd.to_datetime(mot_data['first_use_date'], format='%Y-%m-%d')

#confirm the data types
print(mot_data.dtypes)

test_id                       int64
vehicle_id                    int64
test_date            datetime64[ns]
test_class_id                 int64
test_type                    object
test_result                category
test_mileage                float64
postcode_area                object
make                         object
model                        object
colour                     category
fuel_type                  category
cylinder_capacity           float64
first_use_date       datetime64[ns]
dtype: object


***3. Check for nulls***

In [7]:
#print the shape of the dataset
print(mot_data.shape)

#check for null values
print(mot_data.isnull().sum())

(42216690, 14)
test_id                   0
vehicle_id                0
test_date                 0
test_class_id             0
test_type                 0
test_result               0
test_mileage         324123
postcode_area             0
make                      2
model                    33
colour                    0
fuel_type                 0
cylinder_capacity    279981
first_use_date            0
dtype: int64


***4. Handling nulls:*** 
- test_mileage: This has 324,129 nulls. This variable is an important variable for analyzing the impact of mileage on MOT pass rate. Since the nulls are relatively small (0.77%) compared to the entire dataset, I will be dropping all cars with null as mileage.
- make: Since there are only 2 null values, these will be dropped
- model: Similar to the `make` variable, the number of the nulls is very small (33) so this will be dropped
- cylinder_capacity: with 279,982 nulls and the variable not critical for all analysis, a new category for missing values will be created. All the nulls will be replaced as `0`

In [8]:
#handle null values
mot_data.dropna(subset=['test_mileage'], inplace=True)
mot_data.dropna(subset=['make'], inplace=True)
mot_data.dropna(subset=['model'], inplace=True)
mot_data['cylinder_capacity'].fillna(0, inplace=True)

#check for null values
print(mot_data.isnull().sum())

test_id              0
vehicle_id           0
test_date            0
test_class_id        0
test_type            0
test_result          0
test_mileage         0
postcode_area        0
make                 0
model                0
colour               0
fuel_type            0
cylinder_capacity    0
first_use_date       0
dtype: int64


***5. Incorrect Data Entries***

- Focusing a lot on the Car Make data errors

In [9]:
#print all the unique make
print(mot_data['make'].value_counts())

make
FORD                                 5909505
VAUXHALL                             3937920
VOLKSWAGEN                           3768470
MERCEDES-BENZ                        2230525
BMW                                  2209259
                                      ...   
MINI-EL                                    1
ELDDIS CV20                                1
MITSUBISHE                                 1
GEORGEY BOY                                1
LAND ROVER RANGE ROVER A/BIO 4.4L          1
Name: count, Length: 9812, dtype: int64


- Manually review the car make and start the tedious process of correction (could have used Fuzzy Matching: Apply fuzzy matching to map each car make to the closest common make.)

In [10]:
# Define the mapping of car makes to their standardized forms
make_mapping = {
    'MERCEDES BENZ': 'MERCEDES-BENZ',
    'BENZ': 'MERCEDES-BENZ',
    'MERCEDES': 'MERCEDES-BENZ',
    'HARLEY DAVIDSON': 'HARLEY-DAVIDSON',
    'SMART (MCC)': 'SMART',
    'ISUZU TRUCKS': 'ISUZU',
    'RENAULT TRUCKS': 'RENAULT',
    'GENERAL MOTORS': 'GM',
    'DODGE RAM': 'DODGE',
    'CHRYSLER-JEEP': 'CHRYSLER',
    'ALFA ROMEO': 'ALFA-ROMEO',
    'LAND ROVER': 'LAND-ROVER',
    'ASTON MARTIN': 'ASTON-MARTIN',
    'LONDON TAXIS INT': 'LONDON-TAXIS-INT',
    'ROYAL ENFIELD': 'ROYAL-ENFIELD',
    'ROLLS ROYCE': 'ROLLS-ROYCE',
    'MOTO GUZZI': 'MOTO-GUZZI',
    'ROYAL ALLOY': 'ROYAL-ALLOY',
    'GREAT WALL': 'GREAT-WALL',
    'INDIAN MOTORCYCLE': 'INDIAN-MOTORCYCLE',
    'CI MOTORHOME': 'CI-MOTORHOME',
    'KSR MOTO': 'KSR-MOTO',
    'ERWIN HYMER GROUP UK': 'ERWIN-HYMER',
    'AUSTIN HEALEY': 'AUSTIN-HEALEY',
    'CF MOTO': 'CF-MOTO',
    'VSPA 150': 'VESPA',
    'FERRAI': 'FERRARI',
    'VW': 'VOLKSWAGEN',
}

# Update make_mapping to include the original makes and their standardized forms
for original_make in list(make_mapping.keys()):
    if make_mapping[original_make] not in make_mapping:
        make_mapping[make_mapping[original_make]] = make_mapping[original_make]

# Standardize car makes
mot_data['make'] = mot_data['make'].replace(make_mapping)

#print all the unique make
print(mot_data['make'].value_counts())

make
FORD                                 5909505
VAUXHALL                             3937920
VOLKSWAGEN                           3768588
MERCEDES-BENZ                        2240256
BMW                                  2209259
                                      ...   
HONDA CBR400RR                             1
RELAINT TRIKE                              1
LEAFRANCIS                                 1
A/C                                        1
LAND ROVER RANGE ROVER A/BIO 4.4L          1
Name: count, Length: 9795, dtype: int64


In [11]:
# Filter car makes that have spaces in their name
makes_with_spaces = mot_data[mot_data['make'].str.contains(' ')]['make'].unique()

# Print out the car makes that have spaces in their name
print(makes_with_spaces)

['MITSUBISHI FUSO' 'BMW 520D SE AUTO' 'DODGE (USA)' ... 'TOYOTA VELLFIRE'
 'MERCEDES BENZ ML270 CDI 2.7L' 'LAND ROVER RANGE ROVER A/BIO 4.4L']


In [12]:
# Dictionary mapping prefixes to standardized car makes
make_mapping = {
    'LONDON TAXI': 'LONDON-TAXIS-INT',
    'LONDON TAXIS': 'LONDON-TAXIS-INT',
    'HARLEY-DAVIDSON': 'HARLEY-DAVIDSON',
    'HARLEY': 'HARLEY-DAVIDSON',
    'MERCEDES': 'MERCEDES-BENZ',
    'MERCEDES-BENZ': 'MERCEDES-BENZ',
    'MERCEDEDS' : 'MERCEDES-BENZ',
    'ERWIN HYMER': 'ERWIN-HYMER',
    'LAND ROVER': 'LAND-ROVER',
    'LANDROVER': 'LAND-ROVER',
    'LANDROTER': 'LAND-ROVER',
    'LAND/R': 'LAND-ROVER',
    'LANDOVER': 'LAND-ROVER',
    'RANGEROVER': 'LAND-ROVER',
    'RANG ROVER': 'LAND-ROVER',
    'LAND  ROVER': 'LAND-ROVER',
    'MITSUBISHI': 'MITSUBISHI',
    'MITISUBISHI': 'MITSUBISHI',
    'MITISUBSHI': 'MITSUBISHI',
    'MITSUB ISHI': 'MITSUBISHI',
    'MJTTSUBISHI': 'MITSUBISHI',
    'MTISUBISHI': 'MITSUBISHI',
    'BMW': 'BMW',
    'B M W': 'BMW',
    'DODGE': 'DODGE',
    'CHRYSLER': 'CHRYSLER',
    'SMART': 'SMART',
    'DAF': 'DAF',
    'VESPA': 'VESPA',
    'KURZ': 'KURZ',
    'CHEVROLET': 'CHEVROLET',
    'NGTF': 'NGTF',
    'HERALD': 'HERALD',
    'GM': 'GM',
    'GENRAL MOTORS': 'GM',
    'GENRAL MOTOR' : 'GM',
    'RENAULT': 'RENAULT',
    'LEYLAND': 'LEYLAND',
    'CITROEN': 'CITROEN',
    'CITOEN': 'CITROEN',
    'CITRON' : 'CITROEN',
    'PEUGEOT': 'PEUGEOT',
    'TOYOTA': 'TOYOTA',
    'FORD': 'FORD',
    'MUSTANG': 'FORD',
    'AUDI': 'AUDI',
    'MCLAREN': 'MCLAREN',
    'HYUNDAI': 'HYUNDAI',
    'VOLVO': 'VOLVO',
    'GMC': 'GMC',
    'VOLKSWAGEN': 'VOLKSWAGEN',
    'VOLKSWAGON': 'VOLKSWAGEN',
    'TRANSPORTER   V/W': 'VOLKSWAGEN',
    'VOKSWAGEN': 'VOLKSWAGEN',
    'FIAT': 'FIAT',
    'VAUXHALL': 'VAUXHALL',
    'VAXUHALL': 'VAUXHALL',
    'YAMASAKI': 'YAMASAKI',
    'KIA': 'KIA',
    'HONDA': 'HONDA',
    'PORSCHE': 'PORSCHE',
    'NISSAN': 'NISSAN',
    'ISUZU': 'ISUZU',
    'SEAT': 'SEAT',
    'SKODA': 'SKODA',
    'SWIFT': 'SWIFT',
    'IVECO': 'IVECO',
    'FERRARI': 'FERRARI',
    'LEXUS': 'LEXUS',
    'LIFAN': 'LIFAN',
    'JEEP': 'JEEP',
    'YAMAHA': 'YAMAHA',
    'JARGAUR': 'JARGAUR',
    'JAGUAR': 'JARGAUR',
    'MG ROVER': 'MG',
    'BRISTOL': 'BRISTOL',
    'DUCATO': 'DUCATO',
    'SUZUKI': 'SUZUKI',
    'MAGNUM': 'MAGNUM',
    'SUBARU': 'SUBARU',
    'BENTLEY': 'BENTLEY',
    'MAZDA': 'MAZDA',
    'MAHINDRA': 'MAHINDRA',
    'MG ROVER': 'MG',
    'ROLLS ROYCE': 'ROLLS-ROYCE',
    'ROLLS ROLLS': 'ROLLS-ROYCE',
    'R ROYCE': 'ROLLS-ROYCE',
}

# Function to standardize car makes using the mapping dictionary
def standardize_make(make):
    make_upper = make.upper()
    for prefix, standardized_make in make_mapping.items():
        if make_upper.startswith(prefix):
            return standardized_make
    return make

# Apply the function to the 'make' column
mot_data['make'] = mot_data['make'].apply(standardize_make)

#print all the unique make
print(mot_data['make'].value_counts())

make
FORD                    5909953
VAUXHALL                3938180
VOLKSWAGEN              3769049
MERCEDES-BENZ           2240575
BMW                     2209929
                         ...   
BENELLI TORNADONAKED          1
FS LUBLIN                     1
TOTHTWA                       1
AUTOGRAPH 75-2                1
300 ZX                        1
Name: count, Length: 6945, dtype: int64


- Identify Rare Makes: Identify car makes that have a count less than 100.
- Replace Rare Makes with "Other": Replace these rare makes with the "Other" category.

In [13]:
# Count occurrences of each car make
make_counts = mot_data['make'].value_counts()

# Identify rare makes (those with fewer than 100 occurrences)
rare_makes = make_counts[make_counts < 100].index

# Replace rare makes with "Other"
mot_data['make'] = mot_data['make'].apply(lambda x: 'Other' if x in rare_makes else x)

# Verify the changes
result_withRM = mot_data['make'].value_counts()
print(result_withRM)

make
FORD             5909953
VAUXHALL         3938180
VOLKSWAGEN       3769049
MERCEDES-BENZ    2240575
BMW              2209929
                  ...   
MONTESA              103
OPTARE               103
BETAMOTOR            102
MAHINDRA             102
JAGO                 102
Name: count, Length: 265, dtype: int64


- clean up test results code

In [14]:
#print count for test_results
print(mot_data['test_result'].value_counts())

test_result
P        31982678
F         7871447
PRS       2008052
ABR         26323
ABA          4027
ABRVE           6
Name: count, dtype: int64


In [15]:
""" 
    Create a new column and clean test_result
    Replace P and PRS with PASS, F and ABA with FAIL, ABR and ABRVE with ABORT
"""

mot_data['test_result_clean'] = mot_data['test_result'].replace({'P': 'PASS', 'PRS': 'PASS', 'F': 'FAIL', 'ABA': 'FAIL', 'ABR': 'ABORT', 'ABRVE': 'ABORT'})

# Verify the changes
result_clean = mot_data['test_result_clean'].value_counts()
print(result_clean)

test_result_clean
PASS     33990730
FAIL      7875474
ABORT       26329
Name: count, dtype: int64


***6. Feature Engineering:***
- map postcode to regions
- calculate car age (year of MOT - year of registration)
- categorize mileage

In [16]:
#print unique values
print(mot_data['postcode_area'].unique())

['NW' 'B' 'HA' 'SE' 'UB' 'E' 'N' 'LU' 'BD' 'W' 'SW' 'M' 'LS' 'SR' 'TF'
 'CB' 'DN' 'G' 'PA' 'WF' 'GU' 'CO' 'WS' 'HX' 'EX' 'YO' 'HU' 'GL' 'LA' 'BS'
 'L' 'AB' 'NP' 'OX' 'WN' 'HD' 'TW' 'BH' 'DE' 'EH' 'LL' 'ST' 'PO' 'CV' 'FK'
 'PE' 'ML' 'NR' 'TN' 'SA' 'CA' 'RH' 'TR' 'S' 'NG' 'SK' 'WD' 'NN' 'BA' 'ME'
 'SY' 'SL' 'FY' 'IP' 'KA' 'NE' 'WA' 'CW' 'PR' 'TS' 'CF' 'WV' 'DY' 'DH'
 'BL' 'DL' 'KY' 'DD' 'TA' 'OL' 'RM' 'PH' 'CR' 'SG' 'LE' 'MK' 'IV' 'SN'
 'RG' 'CM' 'HP' 'BB' 'EN' 'AL' 'SO' 'PL' 'LN' 'WR' 'SP' 'BR' 'BN' 'LD'
 'IG' 'KT' 'CT' 'HG' 'DG' 'DA' 'CH' 'TQ' 'SS' 'SM' 'DT' 'TD' 'KW' 'HR'
 'HS' 'XX' 'ZE']


In [17]:
# Define the mapping of postcode areas to town
postcode_mapping = {
    'NW': ('North West London', 'London', 'England'),
    'B': ('Birmingham', 'West Midlands', 'England'),
    'HA': ('Harrow', 'London', 'England'),
    'SE': ('South East London', 'London', 'England'),
    'UB': ('Uxbridge', 'London', 'England'),
    'E': ('East London', 'London', 'England'),
    'N': ('North London', 'London', 'England'),
    'LU': ('Luton', 'East of England', 'England'),
    'BD': ('Bradford', 'Yorkshire and the Humber', 'England'),
    'W': ('West London', 'London', 'England'),
    'SW': ('South West London', 'London', 'England'),
    'M': ('Manchester', 'North West England', 'England'),
    'LS': ('Leeds', 'Yorkshire and the Humber', 'England'),
    'SR': ('Sunderland', 'North East England', 'England'),
    'TF': ('Telford', 'West Midlands', 'England'),
    'CB': ('Cambridge', 'East of England', 'England'),
    'DN': ('Doncaster', 'Yorkshire and the Humber', 'England'),
    'G': ('Glasgow', 'Scotland', 'Scotland'),
    'PA': ('Paisley', 'Scotland', 'Scotland'),
    'WF': ('Wakefield', 'Yorkshire and the Humber', 'England'),
    'GU': ('Guildford', 'South East England', 'England'),
    'CO': ('Colchester', 'East of England', 'England'),
    'WS': ('Walsall', 'West Midlands', 'England'),
    'HX': ('Halifax', 'Yorkshire and the Humber', 'England'),
    'EX': ('Exeter', 'South West England', 'England'),
    'YO': ('York', 'Yorkshire and the Humber', 'England'),
    'HU': ('Hull (Kingston upon Hull)', 'Yorkshire and the Humber', 'England'),
    'GL': ('Gloucester', 'South West England', 'England'),
    'LA': ('Lancaster', 'North West England', 'England'),
    'BS': ('Bristol', 'South West England', 'England'),
    'L': ('Liverpool', 'North West England', 'England'),
    'AB': ('Aberdeen', 'Scotland', 'Scotland'),
    'NP': ('Newport', 'Wales', 'Wales'),
    'OX': ('Oxford', 'South East England', 'England'),
    'WN': ('Wigan', 'North West England', 'England'),
    'HD': ('Huddersfield', 'Yorkshire and the Humber', 'England'),
    'TW': ('Twickenham', 'London', 'England'),
    'BH': ('Bournemouth', 'South West England', 'England'),
    'DE': ('Derby', 'East Midlands', 'England'),
    'EH': ('Edinburgh', 'Scotland', 'Scotland'),
    'LL': ('Llandudno', 'Wales', 'Wales'),
    'ST': ('Stoke-on-Trent', 'West Midlands', 'England'),
    'PO': ('Portsmouth', 'South East England', 'England'),
    'CV': ('Coventry', 'West Midlands', 'England'),
    'FK': ('Falkirk', 'Scotland', 'Scotland'),
    'PE': ('Peterborough', 'East of England', 'England'),
    'ML': ('Motherwell', 'Scotland', 'Scotland'),
    'NR': ('Norwich', 'East of England', 'England'),
    'TN': ('Tonbridge', 'South East England', 'England'),
    'SA': ('Swansea', 'Wales', 'Wales'),
    'CA': ('Carlisle', 'North West England', 'England'),
    'RH': ('Redhill', 'South East England', 'England'),
    'TR': ('Truro', 'South West England', 'England'),
    'S': ('Sheffield', 'Yorkshire and the Humber', 'England'),
    'NG': ('Nottingham', 'East Midlands', 'England'),
    'SK': ('Stockport', 'North West England', 'England'),
    'WD': ('Watford', 'East of England', 'England'),
    'NN': ('Northampton', 'East Midlands', 'England'),
    'BA': ('Bath', 'South West England', 'England'),
    'ME': ('Medway (Rochester)', 'South East England', 'England'),
    'SY': ('Shrewsbury', 'West Midlands', 'England'),
    'SL': ('Slough', 'South East England', 'England'),
    'FY': ('Blackpool', 'North West England', 'England'),
    'IP': ('Ipswich', 'East of England', 'England'),
    'KA': ('Kilmarnock', 'Scotland', 'Scotland'),
    'NE': ('Newcastle upon Tyne', 'North East England', 'England'),
    'WA': ('Warrington', 'North West England', 'England'),
    'CW': ('Crewe', 'North West England', 'England'),
    'PR': ('Preston', 'North West England', 'England'),
    'TS': ('Teesside (Middlesbrough)', 'North East England', 'England'),
    'CF': ('Cardiff', 'Wales', 'Wales'),
    'WV': ('Wolverhampton', 'West Midlands', 'England'),
    'DY': ('Dudley', 'West Midlands', 'England'),
    'DH': ('Durham', 'North East England', 'England'),
    'BL': ('Bolton', 'North West England', 'England'),
    'DL': ('Darlington', 'North East England', 'England'),
    'KY': ('Kirkcaldy', 'Scotland', 'Scotland'),
    'DD': ('Dundee', 'Scotland', 'Scotland'),
    'TA': ('Taunton', 'South West England', 'England'),
    'OL': ('Oldham', 'North West England', 'England'),
    'RM': ('Romford', 'East of England', 'England'),
    'PH': ('Perth', 'Scotland', 'Scotland'),
    'CR': ('Croydon', 'London', 'England'),
    'SG': ('Stevenage', 'East of England', 'England'),
    'LE': ('Leicester', 'East Midlands', 'England'),
    'MK': ('Milton Keynes', 'South East England', 'England'),
    'IV': ('Inverness', 'Scotland', 'Scotland'),
    'SN': ('Swindon', 'South West England', 'England'),
    'RG': ('Reading', 'South East England', 'England'),
    'CM': ('Chelmsford', 'East of England', 'England'),
    'HP': ('Hemel Hempstead', 'East of England', 'England'),
    'BB': ('Blackburn', 'North West England', 'England'),
    'EN': ('Enfield', 'London', 'England'),
    'AL': ('St Albans', 'East of England', 'England'),
    'SO': ('Southampton', 'South East England', 'England'),
    'PL': ('Plymouth', 'South West England', 'England'),
    'LN': ('Lincoln', 'East Midlands', 'England'),
    'WR': ('Worcester', 'West Midlands', 'England'),
    'SP': ('Salisbury', 'South West England', 'England'),
    'BR': ('Bromley', 'London', 'England'),
    'BN': ('Brighton', 'South East England', 'England'),
    'LD': ('Llandrindod Wells', 'Wales', 'Wales'),
    'IG': ('Ilford', 'London', 'England'),
    'KT': ('Kingston upon Thames', 'London', 'England'),
    'CT': ('Canterbury', 'South East England', 'England'),
    'HG': ('Harrogate', 'Yorkshire and the Humber', 'England'),
    'DG': ('Dumfries', 'Scotland', 'Scotland'),
    'DA': ('Dartford', 'South East England', 'England'),
    'CH': ('Chester', 'North West England', 'England'),
    'TQ': ('Torquay', 'South West England', 'England'),
    'SS': ('Southend-on-Sea', 'East of England', 'England'),
    'SM': ('Sutton', 'London', 'England'),
    'DT': ('Dorchester', 'South West England', 'England'),
    'TD': ('Galashiels', 'Scotland', 'Scotland'),
    'KW': ('Kirkwall', 'Scotland', 'Scotland'),
    'HR': ('Hereford', 'West Midlands', 'England'),
    'HS': ('Outer Hebrides', 'Scotland', 'Scotland'),
    'XX': ('Unknown', 'Unknown', 'Unknown'),
    'ZE': ('Shetland', 'Scotland', 'Scotland')
}

# Create DataFrame for mapping
mapping_df = pd.DataFrame(postcode_mapping).T.reset_index()
mapping_df.columns = ['postcode_area', 'town', 'region', 'country']

# Merge with the original dataframe
mot_data = mot_data.merge(mapping_df, on='postcode_area', how='left')

# Display the first few rows of the updated dataframe
mot_data.head()

,test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date,test_result_clean,town,region,country
0,1994821045,838565361,2023-01-02,4,NT,P,179357.0,NW,TOYOTA,PRIUS +,WHITE,HY,1798.0,2016-06-17,PASS,North West London,London,England
1,358005195,484499974,2023-01-01,4,NT,P,300072.0,B,TOYOTA,PRIUS,RED,HY,1500.0,2008-09-13,PASS,Birmingham,West Midlands,England
2,773392437,53988366,2023-01-02,4,NT,PRS,307888.0,HA,TOYOTA,PRIUS,GREY,HY,1497.0,2010-01-15,PASS,Harrow,London,England
3,133665147,606755010,2023-01-02,4,NT,F,65810.0,SE,TOYOTA,PRIUS,SILVER,HY,1497.0,2007-03-28,FAIL,South East London,London,England
4,656743571,606755010,2023-01-02,4,RT,P,65810.0,SE,TOYOTA,PRIUS,SILVER,HY,1497.0,2007-03-28,PASS,South East London,London,England


In [18]:
#print unique values
print(mot_data['region'].unique())

['London' 'West Midlands' 'East of England' 'Yorkshire and the Humber'
 'North West England' 'North East England' 'Scotland' 'South East England'
 'South West England' 'Wales' 'East Midlands' 'Unknown']


In [19]:
#calculate car age in years
mot_data['vehicle_age'] = mot_data['test_date'].dt.year - mot_data['first_use_date'].dt.year

mot_data.head()

,test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date,test_result_clean,town,region,country,vehicle_age
0,1994821045,838565361,2023-01-02,4,NT,P,179357.0,NW,TOYOTA,PRIUS +,WHITE,HY,1798.0,2016-06-17,PASS,North West London,London,England,7
1,358005195,484499974,2023-01-01,4,NT,P,300072.0,B,TOYOTA,PRIUS,RED,HY,1500.0,2008-09-13,PASS,Birmingham,West Midlands,England,15
2,773392437,53988366,2023-01-02,4,NT,PRS,307888.0,HA,TOYOTA,PRIUS,GREY,HY,1497.0,2010-01-15,PASS,Harrow,London,England,13
3,133665147,606755010,2023-01-02,4,NT,F,65810.0,SE,TOYOTA,PRIUS,SILVER,HY,1497.0,2007-03-28,FAIL,South East London,London,England,16
4,656743571,606755010,2023-01-02,4,RT,P,65810.0,SE,TOYOTA,PRIUS,SILVER,HY,1497.0,2007-03-28,PASS,South East London,London,England,16


- According to [moneyraters](https://www.moneyraters.com/car-insurance/average-annual-mileage/), the average mileage per year for UK cars is 7,900.
- Using this as a standard, we can categorize mileage into very-low, low, medium, high and very-high
- Using your guidelines and the average annual mileage of 7,900 miles, we can create categories based on the cumulative mileage over the average age of cars in the UK (approximately 8.6 years). Here’s how the mileage categories could be structured:
    1. Very-Low: Less than the average annual mileage (7,900 miles)
    2. Low: 7,900 to 71,100 miles (1 to 9 years of average mileage)
    3. Medium: 71,101 to 142,200 miles (9 to 18 years of average mileage)
    4. High: 142,201 to 200,000 miles (18 to ~25 years of average mileage)
    5. Very-High: More than 200,000 miles (more than ~25 years of average mileage)

| Catergory | Mileage |
|-----------|---------|
| very-low  | < 7,900 |
| low       | 7,900 - 71,100 |
| medium    | 71,101 - 142,200 |
| high      | 142,201 - 200,000 |
| very-high | > 200,000 |


In [20]:
# Define mileage categories
def categorize_mileage(mileage):
    if mileage < 7900:
        return 'Very-Low'
    elif 7900 <= mileage <= 71100:
        return 'Low'
    elif 71101 <= mileage <= 142200:
        return 'Medium'
    elif 142201 <= mileage <= 200000:
        return 'High'
    else:
        return 'Very-High'

# Apply the categorization
mot_data['mileage_category'] = mot_data['test_mileage'].apply(categorize_mileage)

# Print the updated DataFrame
print(mot_data[['test_id', 'test_mileage', 'mileage_category']])

             test_id  test_mileage mileage_category
0         1994821045      179357.0             High
1          358005195      300072.0        Very-High
2          773392437      307888.0        Very-High
3          133665147       65810.0              Low
4          656743571       65810.0              Low
...              ...           ...              ...
41892528  1748517495       85583.0           Medium
41892529  1705869009      227563.0        Very-High
41892530  1620572037      120115.0           Medium
41892531  1577923551      141891.0           Medium
41892532  1535275065      157901.0             High

[41892533 rows x 3 columns]


***Finally, generate a csv file to be used in PowerBI***

In [21]:
#save mot_data to csv
mot_data.to_csv('mot2023_PBI.csv', index=False, chunksize=500000)

In [22]:
mot_data.head()

,test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date,test_result_clean,town,region,country,vehicle_age,mileage_category
0,1994821045,838565361,2023-01-02,4,NT,P,179357.0,NW,TOYOTA,PRIUS +,WHITE,HY,1798.0,2016-06-17,PASS,North West London,London,England,7,High
1,358005195,484499974,2023-01-01,4,NT,P,300072.0,B,TOYOTA,PRIUS,RED,HY,1500.0,2008-09-13,PASS,Birmingham,West Midlands,England,15,Very-High
2,773392437,53988366,2023-01-02,4,NT,PRS,307888.0,HA,TOYOTA,PRIUS,GREY,HY,1497.0,2010-01-15,PASS,Harrow,London,England,13,Very-High
3,133665147,606755010,2023-01-02,4,NT,F,65810.0,SE,TOYOTA,PRIUS,SILVER,HY,1497.0,2007-03-28,FAIL,South East London,London,England,16,Low
4,656743571,606755010,2023-01-02,4,RT,P,65810.0,SE,TOYOTA,PRIUS,SILVER,HY,1497.0,2007-03-28,PASS,South East London,London,England,16,Low
